In [ ]:
import whisper
from openai import OpenAI
import os
XAI_API_KEY = "your_api_key"

# Whisper 參數
vids_path = "your video path"
model = whisper.load_model("base")  # or "medium" if你已經可以正確載入

# Grok 參數
client = OpenAI(
    api_key=XAI_API_KEY,
    base_url="https://api.x.ai/v1",
)

# Grok 修正字幕文字函式
def grok_chat(prompt):
    completion = client.chat.completions.create(
        model="grok-3-beta",
        messages=[
            {"role": "user", "content": prompt}
        ]
    )
    response = completion.choices[0].message.content
    return response

# Whisper 辨識
print("🎧 辨識影片中...")
result = model.transcribe(vids_path, task="transcribe", verbose=True)

# 儲存成 SRT 字幕檔（修正後字幕）
with open("output_grok.srt", "w", encoding="utf-8") as srt_file:
    for i, segment in enumerate(result["segments"]):
        start = segment["start"]
        end = segment["end"]
        original_text = segment["text"].strip()

        # 格式化時間
        def format_time(seconds):
            hrs = int(seconds // 3600)
            mins = int((seconds % 3600) // 60)
            secs = int(seconds % 60)
            millis = int((seconds - int(seconds)) * 1000)
            return f"{hrs:02}:{mins:02}:{secs:02},{millis:03}"

        # 呼叫 Grok 修正文字
        print(f"🛠 修正字幕段 {i+1}：{original_text}")
        prompt = f"請幫我修正以下字幕內容的語句錯誤，使其更通順：\n\n{original_text}"
        try:
            fixed_text = grok_chat(prompt).strip()
        except Exception as e:
            print(f"⚠️ Grok 呼叫失敗，使用原文：{e}")
            fixed_text = original_text

        # 寫入 SRT 檔
        srt_file.write(f"{i+1}\n")
        srt_file.write(f"{format_time(start)} --> {format_time(end)}\n")
        srt_file.write(f"{fixed_text}\n\n")
